# Embedding
![rag_embedding](figures/rag_embedding.png)

- 분할된 텍스트를 벡터 표현(임베딩 벡터)으로 변환한다.
- LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원하며, 동일한 인터페이스로 사용할 수 있다.
- [임베딩모델의 메서드](https://reference.langchain.com/python/langchain/embeddings/#langchain.embeddings.init_embeddings)

    - **`embed_documents(texts: List[str])`**
        - 여러 문서를 받아 벡터화(임베딩)한다.
        - Context를 벡터화 할 때 사용한다.
    - **`embed_query(text: str)`**
        - 하나의 문자열(문서)을 받아 벡터화한다.
        - Query를 벡터화 할 때 사용한다.


In [1]:
docs = [
        "나는 고양이와 개 중 반려동물로 개를 키우고 싶습니다.",
        "이 강아지 품종은 진도개 입니다. 국제 표준으로 중대형견으로 분류되며 다리가 길어 체고가 높은 편에 속합니다.",
        "日本の市内バスの運賃は主に距離によって決まり、地域やバス会社によって異なる場合があります",                 # 일본의 시내버스 요금은 주로 거리에 따라 결정되며, 지역 및 버스 회사에 따라 다를 수 있습니다.
        "Bus fares in the United States vary from city to city, but are generally around $2.90 for a regular bus.", # 미국의 버스 요금은 도시마다 다르지만, 일반적으로 정기 버스의 경우 2.90달러 정도입니다.
        "광역버스 요금은 일반 3000원, 청소년은 1800원, 어린이는 1500원 입니다.", 
]

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
# OpenAI의 Embedding Model
from langchain_openai import OpenAIEmbeddings

model_name = "text-embedding-3-large"
e_model1 = OpenAIEmbeddings(model=model_name)

In [22]:
!uv pip install sentence-transformers

Resolved 30 packages in 326ms
Prepared 1 package in 98ms
Installed 3 packages in 245ms
 + scikit-learn==1.8.0
 + sentence-transformers==5.2.0
 + threadpoolctl==3.6.0


In [ ]:
# Huggingface Embedding 모델 -> hub: task - NLP - sentence-similarity
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

e_model1 = HuggingFaceEmbeddings(model=model_name)

In [43]:
# Ollama 제공 Embedding 모델 -> 검색에서 embedding 선택
from langchain_ollama import OllamaEmbeddings

model_name = "bge-m3"
e_model1 = OllamaEmbeddings(model=model_name)

In [44]:
embeded_docs = e_model1.embed_documents(docs)
print(type(embeded_docs), len(embeded_docs))

<class 'list'> 5


In [49]:
print(type(embeded_docs[0]), len(embeded_docs[0]))

<class 'list'> 1024


In [50]:
# query = "개와 고양이 중 뭘 더 좋아하나요?" #질문
# query = "요새 버스 요금 얼마야?" #질문
query = "어떤 개를 키우나요?"
embedded_query = e_model1.embed_query(query)

In [10]:
len(embedded_query)

3072

In [36]:
# 질문과 유사한 문서를 찾기.
# 유사도 체크를 위한 방법: 방향기반-코사인유사도, 거리기반-유클리디안(L2), 맨하탄거리계산(L1), 
import numpy as np
def cosine_similarity(vector1, vector2):
    """코사인 유사도 계산
    -1 ~ 1사이 값을 반환
    -1 : 정 반대의 의미
    0: 관계없음
    1: 완벽히 같은 의미
    """
    v1 = np.array(vector1)
    v2 = np.array(vector2)
    return (v1@v2) / (np.linalg.norm(v1)) * np.linalg.norm(v2)

# np.linalg.norm(v1)
# sqrt(sum(v1**2)) # 원소들의 제곱의 합의 제곱근

In [51]:
for i, embedded_doc in enumerate(embeded_docs): # 각 문서들의 embedding vector
    print(f"{i+1}. {cosine_similarity(embedded_query, embedded_doc)}")
    # 1에 가까울수록 유사도가 높음

1. 0.7203114061542991
2. 0.5548700334027135
3. 0.21188058100707544
4. 0.21158872168667323
5. 0.2319182564618554


# 벡터 데이터베이스(Vector Database)

![rag_vector_store](figures/rag_vector_store.png)

- **벡터 데이터베이스란**
-  벡터 데이터베이스는 데이터를 고차원 벡터(임베딩)로 변환하여 저장하고, 벡터 간의 유사도를 기반으로 검색과 관리를 수행하는 특수한 형태의 데이터베이스이다.

- **주요 특징**
  - 텍스트, 이미지, 오디오 등의 비정형 데이터를 수치 벡터로 변환하여 저장
   - 코사인 유사도, 유클리드 거리 등을 이용한 벡터 간 유사도 계산을 통한 검색
   - 근사 최근접 이웃(Approximate Nearest Neighbor, ANN) 알고리즘을 통한 빠른 검색을 지원.

## 벡터 데이터베이스와 딥러닝
- 벡터 데이터베이스는 딥러닝 기술의 발전과 깊은 관련이 있다.
- 딥러닝 모델은 학습 과정에서 데이터의 특징을 추출하는 방법을 함께 학습한다. 충분한 데이터를 학습한 딥러닝 모델은 **데이터의 특성을 설명하는 특성 벡터(feature vector)를 효과적으로 생성**할 수 있다.
- 이때 추출된 특성 벡터는 고차원 데이터(RAW Data)를 저차원 공간에서 표현한 **임베딩 벡터**다.
    - > **임베딩**은 고차원 데이터를 저차원 공간으로 변환하여 표현하는 방법으로, 정보 손실을 최소화하면서 데이터 간의 의미 있는 관계를 벡터 공간에서 유지한다.
- 딥러닝 모델로 추출한 데이터의 특징(feature vector)을 임베딩 공간에 배치하면, 비슷한 데이터는 가까이, 그렇지 않은 데이터는 멀리 배치된다.
- 이러한 특성을 활용하면 임베딩 벡터 간의 거리를 계산해 유사한 데이터를 효과적으로 검색할 수 있다. 벡터 데이터베이스는 이러한 임베딩 벡터의 특성을 기반으로 개발되었다.
- 딥러닝 기술의 발전과 폭넓은 활용으로 임베딩 데이터의 사용이 증가하면서, 이를 저장하고 관리하는 기능에 특화된 데이터베이스에 대한 수요도 증가해 다양한 벡터 데이터베이스가 등장했다.

## LLM과 벡터 데이터베이스
- ChatGPT(LLM)의 등장 이후 벡터 데이터베이스는 폭발적인 주목을 받았다.
- 임베딩 벡터의 유사도를 기반으로 문서를 검색하는 RAG(Relevant Augmented Generation) 기술은 LLM의 환각(할루시네이션) 현상을 줄이고, LLM을 추가 학습하지 않고도 최신 정보를 효율적으로 활용할 수 있는 핵심 기법으로 자리 잡았다.
   


## 벡터 데이터베이스 종류
![img](figures/vector_database.png)

<<https://blog.det.life/why-you-shouldnt-invest-in-vector-databases-c0cd3f59d23c>>

### 주요 벡터 데이터베이스 종류
- **Qdrant**
    - Rust로 개발된 고성능 벡터 검색 엔진으로, 실시간 근사 최근접 이웃 검색을 제공한다.  
    - 추천 시스템에 특화되어 있으며, 벡터 임베딩 저장과 유사도 쿼리를 효율적으로 수행한다.
- **Pinecone**
    - 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 간단한 API를 통해 벡터 데이터를 관리할 수 있다.  
    - 자동 확장성과 고가용성을 제공하며, 실시간 데이터 수집과 유사성 검색에 최적화되어 있다.
    - 가장 쉽게 시작할 수 있는 관리형 서비스를 제공한다.
- **Chroma**
    - 벡터 임베딩을 효율적으로 저장하고 검색할 수 있는 오픈소스 데이터베이스로, AI 및 머신러닝 애플리케이션에 최적화되어 있다.
    - 대규모 임베딩 저장에 최적화되어 있다.
- **FAISS**
    - Facebook AI에서 개발한 고성능 벡터 검색 라이브러리로, 고차원 벡터의 효율적인 유사성 검색을 위해 최적화되어 있다.
    - GPU를 활용해 계산 성능을 높이며, 벡터 양자화 기술을 활용하여 메모리 사용을 최적화한다.
    - 근사 최근접 이웃 검색(ANNS)에 최적화되어 있다.
- **Milvus**
    - 오픈소스 벡터 데이터베이스로, 대규모 벡터 데이터를 효율적으로 저장하고 검색할 수 있다.  
    - 분산 아키텍처를 채택하여 확장성이 뛰어나며, IVF_PQ, DiskANN 등 다양한 인덱싱 알고리즘을 지원한다.
    - 대규모 데이터셋 처리에 가장 적합한 솔루션이다.
- **Weaviate**
    - 오픈소스 벡터 데이터베이스로, 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 저장하고 검색할 수 있다.  
    - GraphQL API를 통해 접근 가능하며, 내장된 머신러닝 모듈을 통해 가장 강력한 의미론적 검색 기능을 제공한다.
- **Elasticsearch**
    - HNSW 알고리즘을 사용하여 벡터 검색을 구현하는 검색 엔진이다.
    - 전통적인 검색 기능과 벡터 검색을 효과적으로 결합할 수 있어, 하이브리드 검색에 가장 적합하다.
- **PGVector**
    - PostgreSQL의 확장 모듈로, 벡터 데이터를 저장하고 유사성 검색을 수행할 수 있게 해준다.  
    - SQL과 통합된 벡터 연산이 가능하며, L2 거리, 코사인 거리, 내적 등 다양한 거리 측정 방식을 지원한다.


# Langchain - Vector Store 연동 
- Langchain은 다양한 벡터 데이터베이스와 연동할 수 있다.
- 벡터 데이터베이스 마다 API가 다르기 때문에, Langchain을 사용하면 동일한 interface로 사용할 수 있다.

## **VectorStore**
- Langchain이 지원하는 모든 벡터 데이터베이스는 **VectorStore** 인터페이스를 구현한다.
- 그래서 Langchain에서는 벡터 데이터베이스를 **Vector Store** 라고 한다.
- https://python.langchain.com/docs/integrations/vectorstores/

### Vector Store 연결
- Vector DB와 연결하는 메소드
- `VectorStore.from_documents()`
  - Document들을 insert 하면서 연결.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - documents: insert할 문서들을 list[Document]로 전달.
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
- `VectorStore()`
  - vector db와 연결만 한다.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
## InMemoryVectorStore
- langchain에서 제공하는 메모리 기반 벡터 데이터베이스이다.
- Data들을 Dictionary를 사용해 메모리에 저장하며, 검색 할 때 코사인 유사도(cosine similarity)를 계산하여 조회한다.

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [54]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Embedding Model 생성
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# VectorStore 생성 -> Embedding 모델을 넣어서 생성.
vectorstore = InMemoryVectorStore(embedding_model)


In [57]:
from langchain_core.documents import Document

d1 = Document(id=1, page_content="Apple, Pear, Watermelon", metadata={"category":"fruit"})
d2 = Document(id=2, page_content="Python, C++, Java", metadata={"category":"it"})
d3 = Document(id=3, page_content="Football, Baseball, Basketball", metadata={"cateogry":"sports"})

# VectorStore에 데이터를 넣기. - add_documents() 리스트로 묶어서 전달.
vectorstore.add_documents([d1, d2, d3])

# VectorStore를 생성(연결) 하면서 문서들을 넣기(upsert)
# InMemoryVectorStore.from_documents(
#     documents=[d1, d2, d3],
#     embedding=embedding_model
# )

['1', '2', '3']

In [68]:
# 검색 - 유사도 기반 검색
query = "Messi"
result = vectorstore.similarity_search(query=query, k=3) # k: 유사도 높은 순서대로 문서 k개를 반환.
# result
for i, r in enumerate(result):
    print(f"{i+1}. {r.page_content}")

1. Football, Baseball, Basketball
2. Python, C++, Java
3. Apple, Pear, Watermelon


In [69]:
# 검색 - 유사도 기반 검색
query = "SQL"
result = vectorstore.similarity_search_with_score(query=query, k=3) # 유사도 점수까지 반환
# tuple - (검색한 Document, 유사도 점수)
for doc, score in result:
    print(doc.page_content, score)
# result
# for i, r in enumerate(result):
#     print(f"{i+1}. {r.page_content}")

Python, C++, Java 0.3440530439392729
Football, Baseball, Basketball 0.2636065886635491
Apple, Pear, Watermelon 0.12970742236928104


# 실습
1. text loading
2. text split
3. embedding + vector store(InMemoryVectorStore)에 저장
4. query(질의)

In [86]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

path = "data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name = "gpt-5-mini", # gpt-5-mini 에서 사용한 tokenizer 기준으로 한다.
    chunk_size = 500,
    chunk_overlap = 20
)
docs = loader.load_and_split(splitter)
len(docs)


36

In [96]:
d = list()
for idx, doc in enumerate(docs):
    d.append(Document(id=idx, page_content=doc.page_content, metadata=doc.metadata))
d
# d1 = Document(id=1, page_content="Apple, Pear, Watermelon", metadata={"category":"fruit"})

[Document(id='0', metadata={'source': 'data/olympic.txt'}, page_content='올림픽\n올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.'),
 Document(id='1', metadata={'source': 'data/olympic.txt'}, page_content='또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사

In [89]:
docs[2].page_content

'올림픽은 거의 모든 국가가 참여할 정도로 규모가 커졌다. 하계 올림픽은 33개의 종목과 약 400개의 세부종목에서 13,000명이 넘는 선수들이 겨루고 그중 각 종목별 1, 2, 3위는 각각 금/은/동을 수여받는다. 전 세계 언론에서 각각 4년마다 열리는 올림픽 경기를 중계하기 때문에 이름 없는 선수가 개인적, 국가적, 세계적으로 명성을 얻을 수 있는 기회가 된다. 이와 더불어 올림픽 경기는 개최지와 개최국에게도 전 세계에 그 이름을 널리 알리는 좋은 기회가 된다.'

In [76]:
print(docs[0].page_content)

올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.
또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20세기의 변화하는 경제, 정

In [77]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Embedding Model 생성
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# VectorStore 생성 -> Embedding 모델을 넣어서 생성.
vectorstore = InMemoryVectorStore(embedding_model)


In [98]:
vectorstore.add_documents(d)

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35']

In [ ]:
# 검색 - 유사도 기반 검색
query = "근대올림픽"
result = vectorstore.similarity_search_with_score(query=query, k=3) # 유사도 점수까지 반환
# tuple - (검색한 Document, 유사도 점수)
for doc, score in result:
    print(score, doc.page_content)
# result
# for i, r in enumerate(result):
#     print(f"{i+1}. {r.page_content}")

고대올림픽
고대의 올림픽 경기(올림피아 경기)는 고대 그리스의 여러 도시 국가의 대표선수들이 모여 벌인 일련의 시합이었으며, 육상 경기가 주 종목이지만 격투기와 전차 경기도 열렸다. 그리고 패배하면 죽기도 하였다. 고대 올림픽의 유래는 수수께끼로 남아있다. 잘 알려진 신화로는 헤라클레스와 그의 아버지인 제우스가 올림픽의 창시자였다는 것이다. 전설에 따르면 이 경기를 최초로 '올림픽'이라고 부르고, 4년마다 대회를 개최하는 관례를 만든 사람이 헤라클레스라고 한다. 어떤 전설에서는 헤라클레스가 이른바 헤라클레스의 12업을 달성한 뒤에 제우스를 기리고자 올림픽 경기장을 지었다고 한다. 경기장이 완성되자 헤라클레스는 일직선으로 200 걸음을 걸었으며, 이 거리를 "스타디온"이라 불렀는데, 후에 이것이 길이 단위인 '스타디온'(그리스어: στάδιον → 라틴어: 영어: stadium)이 되었다. 또 다른 설로는 '올림픽 휴전'(그리스어: ἐκεχειρία 에케케이리아[*])이라는 고대 그리스의 관념이 최초의 올림피아 경기와 관련이 있다고 한다. '올림픽 휴전'이란 어느 도시 국가라도 올림피아 경기 기간 중에 다른 나라를 침범하면 그에 대한 응징을 받을 수 있다는 뜻으로, "올림픽 기간에는 전쟁하지 말 것"으로 요약할 수 있다. 0.5047465973747226
근대올림픽
고대 올림피아 경기를 제대로 구현한 최초의 시도는 혁명 시대의 프랑스에서 1796년부터 1798년까지 3년동안 실시했던 프랑스 국내 올림픽인 '공화국 올림픽'(L'Olympiade de la République)이었다. 이 대회의 종목 중에는 고대 그리스 올림피아 경기 때 행한 일부 종목도 있었다. 특히 1798년 공화국 올림픽 대회는 미터법을 최초로 스포츠에 도입시킨 대회이기도 하다. 이후 52년뒤인 1850년에는 잉글랜드 슈롭셔주의 웬록에서 올림픽급의 대회가 열리기 시작하였다. 이 대회는 1859년에 아테네에서 열렸을 때 웬록 올림픽으로 명칭이 변경되었으며 지금도 열리고 있다. 브룩스 박사는 18

이 위는 내가한거. 밑에는 정답

In [103]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from dotenv import load_dotenv
load_dotenv()

path = "data/olympic.txt"
# 1. 문서 로드 + Split
loader = TextLoader(path, encoding="utf-8")
# 2. splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=50
)

docs = loader.load_and_split(splitter)
docs = [doc for doc in docs if len(doc.page_content) > 10] # 10글자 이내는 제거
len(docs)

# 3. vectorstore에 연결 + 저장
## 1. embedding model
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
## 2. VectorStore 생성
vectorstore = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embedding
)

In [128]:
# 질의
# query = "올림픽과 관련된 논란은 어떤것이 있나요?"
query = "올림픽이 취소된 경우가 있어? 있다면 몇년도에 어떤 이유로 취소됐는지 알려줘"

result_docs = vectorstore.similarity_search_with_score(query, k=5)
for doc, score in result_docs:
    print(">>>", score, doc, sep="|||")

>>>|||0.49579624673845274|||page_content='쿠베르탱의 생각과는 달리, 올림픽이 세계에 완벽한 평화를 가져다주지는 못했다. 실제로 제1차 세계대전으로 인해 독일 베를린에서 열리기로 했던 제6회 1916년 하계 올림픽이 취소되었고, 제2차 세계대전 때는 일본 도쿄에서 열리기로 했던 제12회 1940년 하계 올림픽, 삿포로에서 열리기로 했던 1940년 동계 올림픽, 영국 런던에서 열리기로 했던 제13회 1944년 하계 올림픽, 이탈리아 코르티나담페초에서 열릴 예정인 1944년 동계 올림픽이 취소되었다. 베이징에서 열린 2008년 하계 올림픽 개막식날 조지아와 러시아 간의 2008년 남오세티아 전쟁이 일어나기도 했다. 부시 대통령과 푸틴 대통령이 이 올림픽을 보러 왔으며 중국 주석인 후진타오가 주최한 오찬에 참석해서 이 현안에 대해 논의하기도 했다. 조지아 대표인 니노 살루크바체와 러시아 대표인 나탈리야 파데리나가 여자 10m 공기권총 경기에서 각각 동메달과 은메달을 땄을 때 이 일은 베이징 올림픽의 유명한 사건 중 하나로 남게 되었다. 살루크바체와 파데리나는 시상식이 끝난 뒤 서로 포옹을 하며 국적에 상관없이 기쁨을 나누었다.' metadata={'source': 'data/olympic.txt'}
>>>|||0.44715523653069533|||page_content='올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르

In [125]:
# 질문 + 검색한 결과 context로 LLM에게 질의
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    template="""# instruction
    당신은 QA 전문 assistant입니다.
    질문에 대해 주어진 context를 기반으로 답을 해주세요.
    context에 질문과 관련된 내용이 없을 경우 '주어진 정보로는 답을 알 수 없습니다.' 라고 답을 하세요.
    context에 없는 내용으로 답변을 만들지 마세요.

    # context
    {context}

    # 질문
    {query}

    # Output indicator
    - 답변을 context 어느부분을 참조하였는지 각주를 달아줘.

    예)
    '''markdown
    # 답변
    최종답변내용

    # 참조내용
    context에서 참조한 내용
    '''
    """
)

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

chain = prompt | model | parser

In [129]:
# VectorDB에서 조회한 결과중 page_content값만 추출 -> prompt의 context 주입
context = '\n\n'.join(doc[0].page_content for doc in result_docs)
# context
result = chain.invoke({"query":query, "context":context})

In [130]:
print(result)

```markdown
# 답변
네, 취소된 경우가 있습니다.

- 1916년 제6회 하계 올림픽(베를린) — 제1차 세계대전으로 취소되었다.[1]  
- 1940년 제12회 하계 올림픽(도쿄) — 제2차 세계대전으로 취소되었다.[2]  
- 1940년 동계 올림픽(삿포로) — 제2차 세계대전으로 취소되었다.[2]  
- 1944년 제13회 하계 올림픽(런던) — 제2차 세계대전으로 취소되었다.[2]  
- 1944년 동계 올림픽(코르티나담페초) — 제2차 세계대전으로 취소되었다.[2]

# 참조내용
[1] "실제로 제1차 세계대전으로 인해 독일 베를린에서 열리기로 했던 제6회 1916년 하계 올림픽이 취소되었고,"  

[2] "제2차 세계대전 때는 일본 도쿄에서 열리기로 했던 제12회 1940년 하계 올림픽, 삿포로에서 열리기로 했던 1940년 동계 올림픽, 영국 런던에서 열리기로 했던 제13회 1944년 하계 올림픽, 이탈리아 코르티나담페초에서 열릴 예정인 1944년 동계 올림픽이 취소되었다."
```


## MMR(최대 한계 관련성-Maximal Marginal Relevance) 알고리즘 적용
최대 한계 관련성(Maximal Marginal Relevance, MMR) 알고리즘은 정보 검색 및 요약에서 검색 결과의 **관련성**과 **다양성**을 동시에 고려하여 최적의 결과를 제공하는 방법이다. 
이 알고리즘은 사용자 쿼리와의 관련성을 최대화하면서도 중복 정보를 최소화하여 다양한 정보를 제공하는 것을 목표로 한다.

1. **관련성과 다양성의 균형 조절**: MMR은 사용자 쿼리와 문서 간의 유사성 점수와 이미 선택된 문서들과의 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한다. 이를 통해 관련성이 높으면서도 중복되지 않는 문서를 선택한다.

2. **수학적 정의**
   $$
   \text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')
   $$

   - $\text{Sim}(d, Q)$: 문서 $d$와 쿼리 $\text{Q}$ 사이의 유사성. (문서 유사성 계산)
   - $\max_{d' \in D'} \text{Sim}(d, d')$: 문서 $d$와 이미 선택된 문서 집합 $D'$ 중 가장 유사한 문서와의 유사성. (문서 다양성 계산)
   - $\lambda$: 유사성과 다양성의 중요도를 조절하는 매개변수(parameter)
3. **적용 분야**: MMR은 정보 검색, 추천 시스템, 문서 요약 등에서 활용된다. 특히 LLM 검색에서 성능 향상이 입증되었다.

### `vectorStore.max_marginal_relevance_search()` 메소드
  - MMR 알고리즘을 적용한 검색을 수행한다.
  - **파라미터**
    - **query**: 사용자로부터 입력받은 검색 쿼리
    - **k**: 최종적으로 선택할 문서의 수
    - **fetch\_k**: MMR 알고리즘 적용 시 고려할 상위 문서의 수
    - **lambda_mult**: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절하는 매개변수. $\lambda = 1$이면 유사성만 고려하고, $\lambda = 0$이면 다양성만을 최대화한다.
    - **filter**: 검색 결과를 필터링할 조건을 지정한다.


In [133]:
query = "IOC는 어떤 기구입니까??"

mmr_result = vectorstore.max_marginal_relevance_search(
    query=query,
    k=5, 
    fatech_k=10,    # 다양성 계산을 위해 상위 몇개의 문서르 볼지.
    lambda_mult=0.5 # 관련성, 다양성을 반반씩.
)

for dc in mmr_result:
    print(doc.page_content[:50])
    print("-"*50)

올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡
--------------------------------------------------
올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡
--------------------------------------------------
올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡
--------------------------------------------------
올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡
--------------------------------------------------
올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡
--------------------------------------------------
